# Lab-based Group Assignment 1:
## Prediction of Loyalty of Credit Applications for Risky Customers

<div class="LI-profile-badge"  data-version="v1" data-size="large" data-locale="en_US" data-type="horizontal" data-theme="light" data-vanity="drsalihtutun"><a class="LI-simple-link" href='https://www.linkedin.com/in/drsalihtutun/en-us?trk=profile-badge'>Salih Tutun, PhD</a></div>

![Imgur](https://i.imgur.com/r1U9dHD.png)

### Business Problem

<font color='green'>Understand the uncertain loyalty level (for payment difficulties) of customers given his/her background information collected in the application process. </font>

### Business Objectives

This application aims to identify patterns that indicate <font color='green'>if a customer has difficulty paying their installments</font> which may be used for taking actions such as denying the loan, reducing the amount of loan, lending (to risky applicants) at a higher interest rate, etc. And this will ensure that the consumers capable of repaying the loan will not be rejected.


![Imgur](https://i.imgur.com/tyqhhLR.png)

**Credit:** Investopedia

In [13]:
#Mount with Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).




### <font color='red'>**Q1: Please explore the given dataset** </font>

Please read the dataset.

In [14]:
# Import the dataset named 'credit_data.csv'
# The data is kind of large. Please only load the first 1000 rows.
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Deep Learning/Credit_Card_Dataset.csv', nrows = 1000).drop("Car Owner",axis=1)
data.head()

# Target = 1 => Fraud Customer

,Mobile Phone,Work Phone,Correct Phone Number,City Rating,Region Rating,House Owner,Income,Document,Target
0,1,1,1,2,2,1,1,1,0
1,1,1,1,1,1,0,0,1,0
2,1,1,1,2,2,1,0,0,1
3,1,1,1,2,2,1,0,1,0
4,1,1,1,2,2,1,0,0,0


In [15]:
#Column Description is shown below
pd.read_csv('/content/drive/MyDrive/Deep Learning/Credit Card Column Description.csv', sep = ';', index_col = 0)

,Description
Column Name,
Mobile Phone,"(0,1) 1: Client provided Mobile phone"
Work Phone,"(0,1) 1: Client provided Work phone"
Correct Phone Number,"(0,1) 1: Phone numbers provided are reachable"
City Rating,"(1,2,3) 1 is the best"
Region Rating,"(1,2,3) 1 is the best"
House Owner,"(0,1) 1: Client owns certain real estate"
Car Owner,"(0,1) 1: Client owns a car"
Income,Income of the client
Document,"(0,1) 1: Client provided additional document"


In [16]:
# Please read house owner, income, document and target columns.
#View the head of the data
#We only look at three most related factors (House Owner, Income and Document) and the dependent variable Target
data_adj = data.loc[:,["House Owner", "Income", "Document", "Target"]]
data_adj.head(10)

,House Owner,Income,Document,Target
0,1,1,1,0
1,0,0,1,0
2,1,0,0,1
3,1,0,1,0
4,1,0,0,0
5,1,0,1,0
6,1,0,0,1
7,1,0,1,1
8,1,0,1,0
9,1,0,0,0


In [17]:
#View the main statistics of the data columns
#You may use DataFrame.description() method
data_adj.describe()

,House Owner,Income,Document,Target
count,1000.000000,1000.000000,1000.00000,1000.000000
mean,0.707000,0.070000,0.72300,0.335000
std,0.455366,0.255275,0.44774,0.472227
min,0.000000,0.000000,0.00000,0.000000
25%,0.000000,0.000000,0.00000,0.000000
50%,1.000000,0.000000,1.00000,0.000000
75%,1.000000,0.000000,1.00000,1.000000
max,1.000000,1.000000,1.00000,1.000000


In [18]:
#View the datatype of each column
#You may use the DataFrame.info() method
data_adj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   House Owner  1000 non-null   int64
 1   Income       1000 non-null   int64
 2   Document     1000 non-null   int64
 3   Target       1000 non-null   int64
dtypes: int64(4)
memory usage: 31.4 KB


### <font color='red'>**Q2:Please prepare your dataset for the model** </font>

In [19]:
#Split the data into training set (80%) and testing set (20%)
import numpy as np

#Get all index first
all_index = np.array(data_adj.index)


#Randomly select 80% of the index as training index
#You may use np.random.choice() to sample from a set
train_index = np.random.choice(all_index,size=int(1000*0.8),replace=False)


#Get the rest as the test index
#You may use np.setdiff1d() to get the complement set
test_index = np.setdiff1d(all_index, train_index)


#Split the data using the split index
train_data = data_adj.loc[train_index,]
test_data = data_adj.loc[test_index,]
test_data

,House Owner,Income,Document,Target
0,1,1,1,0
5,1,0,1,0
16,0,0,1,0
20,1,0,0,1
25,1,0,1,0
...,...,...,...,...
975,1,0,0,1
982,1,0,0,0
984,1,0,1,1
985,0,0,1,0


### <font color='red'>**Q3:Please define your activation functions and initialize the weights of the model** </font>

- Sub-Question:
Think what happens when deriv = False vs when deriv = True.

In [20]:
#Build your own neural network below and print out the MSE
#If you meet difficulty, feel free to refer to "M1.6. Building Neural Networks.ipynb"

#Define a non-linear function first
#Don't forget to define the first order derivative of your non-linear function
#To combine them together, we may add a parameter in the non-linear function

import numpy as np

def relu(x,deriv=False):
    if(deriv==True):

        #return the first order derivative
        return x>0

    #return the function
    return (x>0)*x

def sigm(x,deriv=False):
    if(deriv==True):
      y=1/(1+np.exp(-x))
      return y*(1-y)

    return 1/(1+np.exp(-x))

 **If deriv = False:**

 relu: return x, with range 0 <= x

 sigm: return 1/(1+exp^-x), also with range 0 ~ 1

 **If deriv = True:**

 relu: return 1 if x > 0; return 0 otherwise

 sigm: return (1/(1+exp^-x))*(1-(1/(1+exp^-x)))

 **conclusion:**

 "deriv = False" returns the output of the activation functions

 "deriv = True" returns the first derivative of the activation functions

![Imgur](https://i.imgur.com/9TVc4LU.png)

Sub-Question: What would be X.shape() and y.shape() and why? Think about train dataset x variables and y variable.


In [29]:
#What is your X value? House Owner, Income and Document. Those three factors are related to whether one can pay the installments.
X = train_data.iloc[:,0:-1].values  # we can call X as a0 as well.

#What is your y value? Target. That is whether one can pay the installments.
y = train_data.iloc[:,-1].values

# You need to reshape your y data into a 2-dimensional array
y = np.reshape(y,(train_data.shape[0],1))


# randomly initialize our weights with as 2x-1
# w0 = 2*np.random.random((input,hidden)) - 1
# w1 = 2*np.random.random((hidden,output)) - 1
w0 = 2*np.random.random((3,10)) - 1
w1 = 2*np.random.random((10,1)) - 1

for iter in range(10000):

    # forward propagation
    a0 = X
    a1 = sigm(np.dot(a0,w0))

    # how much did we miss?
    a1_error = y - a1

    # multiply how much we missed by the
    # slope of the sigmoid at the values in l1
    a1_delta = a1_error * sigm(a1,True)

    # update weights
    w0 += np.dot(a0.T,a1_delta)

    # MSE
    mse = np.mean(np.square(a1_error))

w0, w1, mse

(array([[ 42.93884605,  19.93509859, -28.71741596,  31.32088191,
         -43.94044741,  10.4937126 , -51.53788493, -59.98614046,
         -51.30073067,  -4.50839724],
        [ -5.59960862,  -6.57491682, -10.02052115,  -1.22169695,
          -7.699479  ,  -6.96328623, -11.14527251, -11.78807514,
         -17.14942691, -14.56593025],
        [ 21.73326892,  10.39241856, -34.92751238,  27.50101679,
         -45.60997661,  -9.61418453, -49.45946536, -58.53628412,
         -54.96474981, -20.67108656]]),
 array([[-0.35703047],
        [-0.23119599],
        [-0.34739501],
        [ 0.85985429],
        [-0.740192  ],
        [ 0.56311114],
        [-0.66958618],
        [ 0.90754835],
        [-0.73425065],
        [ 0.4122474 ]]),
 0.475872233808028)

In [30]:
X.shape, y.shape

((800, 3), (800, 1))

**X.shape: The shape of X is (m, 3)**

train_data.iloc[:, 0:-1] selects all rows and all columns except the last one.

So, X includes: House Owner, Income, and Document.

X is an 3-dimensional array with a shape of (m,3), where m is the number of rows

**y.shape: The shape of y is (m, 1)**

train_data.iloc[:, -1] selects all rows and only the last column, which is the target variable indicating whether one can pay the installments.

y is an 1-dimensional array with a shape of (m,1), where m is the number of rows

**m=800**

the number of rows = the number of training data

Thus, m = 800 (1000*.8)

**X.shape = (800, 3), y.shape = (800, 1)**


![Imgur](https://i.imgur.com/85DTrAa.png)

### <font color='red'>**Q4:Please train your neural networks** </font>

In [31]:

#Write a long enough loop to train your model and minimize the cost

for j in range(60000):

    #forward propagation begins#
    # Feed forward through layers 0, 1, and 2
    a0 = X
    a1 = sigm(np.dot(a0,w0))
    a2 = sigm(np.dot(a1,w1))

    # how much did we miss the target value?
    a2_error= y - a2
    #forward propagation ends#

    # in what direction is the target value?
    # were we really sure? if so, don't change too much.
    # Backpropagation begins#
    a2_delta = a2_error*sigm(a2,deriv=True)

    # how much did each a1 value contribute to the a2 error (according to the weights)?
    a1_error = a2_delta.dot(w1.T)

    # in what direction is the target l1?
    # were we really sure? if so, don't change too much.
    a1_delta = a1_error * sigm(a1,deriv=True)
    # the update of weights
    w1 += a1.T.dot(a2_delta)
    w0 += a0.T.dot(a1_delta)
    # Backpropagation ends#
#Let's convert the probability into binary result
#1 means correct prediction; 0 means mistake
train_result = [1 if np.abs(e) < 0.5 else 0 for e in a2_error]

#what is our training accuracy
np.mean(train_result)


<ipython-input-20-f00553b84685>:24: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


0.03875

### <font color='red'>**Q5:Please test your model with test dataset** </font>

In [ ]:
def my_net(w0, w1, test_data):

    #Get your test data first
    X = test_data.iloc[:,0:3].values
    y = test_data.iloc[:,3].values
    y = np.reshape(y,(test_data.shape[0],1))

    #Input your data in the first layer
    a0 = X

    #Caculate the second layer
    a1 = sigm(np.dot(a0,w0))

    #Caculate the third layer
    a2 = sigm(np.dot(a1,w1))

    #What is your error?
    a2_error = y - a2

    #Finally return your error
    test_result = [1 if np.abs(e) < 0.5 else 0 for e in a2_error]
    return np.mean(test_result)

#Print the score
my_score = my_net(w0, w1, test_data)
print("Test Accuracy: " + str(my_score))

Test Accuracy: 0.22


<ipython-input-21-989c1d878d1e>:24: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


In [33]:
def my_net(w0, w1, test_data):

    #Get your test data first
    X = test_data.iloc[:,0:3].values
    y = test_data.iloc[:,3].values
    y = np.reshape(y,(test_data.shape[0],1))

    #Input your data in the first layer
    a0 = X

    #Caculate the second layer
    a1 = sigm(np.dot(a0,w0))

    #Caculate the third layer
    a2 = sigm(np.dot(a1,w1))

    a2_error = y - a2
    mse = np.mean(np.square(a2_error))

    test_result = [1 if np.abs(e) < 0.5 else 0 for e in a2_error]
    accuracy = np.mean(test_result)

    # Return both accuracy and MSE
    return mse

# Print the accuracy and MSE for the test data
test_mse = my_net(w0, w1, test_data)
print(f"Test MSE: {test_mse}")

Test MSE: 0.28374917278058354


<ipython-input-20-f00553b84685>:24: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


Note: Accuracy could be change for each run because we have random intial weights.

If you have questions, please contact with me.

Salih Tutun, PhD

salihtutun@wustl.edu